In [1]:
# Check P2M, M2P and M2M
import numpy as np
from pcm_fmm import pcm_fmm
p = 10
ngrid = 6
vscales, w, grid, vgrid = pcm_fmm.init_globals(p, ngrid)
src = np.array([0.5,0.4,0.3], dtype=np.float64)
dst = np.array([1, 1.1, 2.2], dtype=np.float64)
p2p = np.linalg.norm(src-dst)**-1
sph1_c = np.zeros(3)
sph1_r = 1
sph1_coef = pcm_fmm.fmm_p2m(src-sph1_c, sph1_r, p, vscales)
m2p = pcm_fmm.fmm_m2p(dst-sph1_c, sph1_r, p, vscales, sph1_coef)
print("|| P2M + M2P - P2P ||  /  || P2P || =", abs((m2p-p2p)/p2p))
sph2_c = -np.ones(3)
sph2_r = sph1_r + np.linalg.norm(sph2_c-sph1_c)
sph2_coef = pcm_fmm.fmm_p2m(src-sph2_c, sph2_r, p, vscales)
sph2_coef_m2m = np.zeros_like(sph2_coef)
pcm_fmm.fmm_m2m_baseline(sph1_c-sph2_c, sph1_r, sph2_r, p, vscales, sph1_coef, sph2_coef_m2m)
print("|| P2M(1) + M(1)2M(2) - P2M(2) ||  /  || P2M(2) || =",
      np.linalg.norm(sph2_coef_m2m-sph2_coef) / np.linalg.norm(sph2_coef))

|| P2M + M2P - P2P ||  /  || P2P || = 1.2811025058542572e-07
|| P2M(1) + M(1)2M(2) - P2M(2) ||  /  || P2M(2) || = 3.244277800823038e-16


In [2]:
# Check M2L, L2P and L2L
import numpy as np
from pcm_fmm import pcm_fmm
pl = 10
pm = 10
ngrid = 6
vscales, w, grid, vgrid = pcm_fmm.init_globals(pl+pm, ngrid)
src = np.array([0.5,0.4,0.3], dtype=np.float64)
dst = np.array([1, 1.1, 2.2], dtype=np.float64)
p2p = np.linalg.norm(src-dst)**-1
# Compute multipole coefficients
sph1_c = np.zeros(3)
sph1_r = 2
sph1_coef = pcm_fmm.fmm_p2m(src-sph1_c, sph1_r, pm, vscales[:(pm+1)**2])
# Compute local expansion
sph2_c = np.array([1, 1, 2], dtype=np.float64)
sph2_r = 1.3
sph2_coef = np.zeros((pl+1)*(pl+1))
pcm_fmm.fmm_m2l_baseline(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef)
l2p = pcm_fmm.fmm_l2p(dst-sph2_c, sph2_r, pl, vscales[:(pl+1)**2], sph2_coef)
print("|| P2M + M2L + L2P - P2P ||  /  || P2P || =", abs((l2p-p2p)/p2p))
sph3_c = np.array([1, 1, 2], dtype=np.float64)
sph3_r = 0.6
sph3_coef = np.zeros_like(sph2_coef)
pcm_fmm.fmm_l2l_baseline(sph2_c-sph3_c, sph2_r, sph3_r, pl, vscales[:(pl+1)**2], sph2_coef, sph3_coef)
l2l2p = pcm_fmm.fmm_l2p(dst-sph3_c, sph3_r, pl, vscales[:(pl+1)**2], sph3_coef)
print("|| L(1)2L(2) + L(2)2P - L(1)2P ||  /  || L(1)2P || =", abs((l2l2p-l2p)/l2p))
sph3_c = np.array([1.1, 1.1, 2.1], dtype=np.float64)
sph3_r = 0.6
sph3_coef = np.zeros_like(sph2_coef)
pcm_fmm.fmm_l2l_baseline(sph2_c-sph3_c, sph2_r, sph3_r, pl, vscales[:(pl+1)**2], sph2_coef, sph3_coef)
l2l2p = pcm_fmm.fmm_l2p(dst-sph3_c, sph3_r, pl, vscales[:(pl+1)**2], sph3_coef)
print("|| L(1)2L(3) + L(3)2P - L(1)2P ||  /  || L(1)2P || =", abs((l2l2p-l2p)/l2p))

|| P2M + M2L + L2P - P2P ||  /  || P2P || = 1.2813589395338697e-07
|| L(1)2L(2) + L(2)2P - L(1)2P ||  /  || L(1)2P || = 0.0
|| L(1)2L(3) + L(3)2P - L(1)2P ||  /  || L(1)2P || = 6.946662007989605e-16


In [1]:
import numpy as np
from time import time
from pcm_fmm import pcm_fmm
from mydx import mydx
mydx.init()
nbasis, ngrid, nsph = mydx.get_sizes()
csph, rsph = mydx.get_spheres(nsph)
ui = mydx.get_ui(nsph, ngrid)
p = int(nbasis**0.5)-1
print("nsph = {}\np = {}\nngrid = {}".format(nsph, p, ngrid))
vscales, w, grid, vgrid = pcm_fmm.init_globals(p, p, ngrid)
ind = np.arange(1, nsph+1, dtype=np.int32)
cluster, children, parent, cnode, rnode, snode = pcm_fmm.tree_init(csph, rsph, ind)

nsph = 141
p = 6
ngrid = 110


In [2]:
coef_sph = np.random.randn(nbasis, nsph)
t0 = time()
coef_out = pcm_fmm.pcm_matvec_grid_treecode(csph, rsph, grid, w, vgrid, ui, p, vscales[:(p+1)*(p+1)], ind, \
                                   cluster, children, cnode, rnode, snode, coef_sph)
print("FMM: ", time()-t0, " seconds")

FMM:  0.1586768627166748  seconds


In [3]:
t0 = time()
coef_ddpcm = mydx.ddpcm_dx(coef_sph)
print("ddPCM: ", time()-t0, " seconds")
print(np.linalg.norm(coef_ddpcm-coef_out), np.linalg.norm(coef_ddpcm))

ddPCM:  0.20003199577331543  seconds
0.18157080019361305 161.10027608711601


In [5]:
lwork = nsph*100
iwork = np.zeros(1, dtype=np.int32)
jwork = np.zeros(1, dtype=np.int32)
work = np.zeros((3,lwork), dtype=np.int32, order='F')
nnfar, nfar, nnnear, nnear = pcm_fmm.tree_get_farnear_work(children, cnode, rnode, iwork, jwork, work)
if(iwork[0] != jwork[0]+1):
    printf("LWORK IS TOO SMALL")
else:
    sfar, far, snear, near = pcm_fmm.tree_get_farnear(jwork[0], work, nnfar, nfar, nnnear, nnear)
    pcm_fmm.pcm_matvec_grid_fmm(csph, rsph, grid, w, vgrid, ui, p, p, vscales, ind, cluster, \
                            children, cnode, rnode, snode, sfar, far, snear, near, coef_sph)